In [3]:
import re
import nltk
import gensim
import gensim.corpora as corpora
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel
import warnings
warnings.filterwarnings("ignore")

In [4]:
dataset = pd.read_csv("data/tweets_01-08-2021.csv")

In [5]:
trump = dataset

In [6]:
print(len(dataset)) #the length of the data
print(type(dataset)) # the type of variable the data is stored in
print(dataset[:1]) # the first instance of the content within the data

56571
<class 'pandas.core.frame.DataFrame'>
                  id                                               text  \
0  98454970654916608  Republicans and Democrats have both created ou...   

  isRetweet isDeleted     device  favorites  retweets                 date  \
0         f         f  TweetDeck         49       255  2011-08-02 18:07:48   

  isFlagged  
0         f  


In [7]:
dataset.head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f


In [8]:
full_train = pd.DataFrame()
full_train['text'] = dataset['text']

In [9]:
tweets = full_train

In [10]:
tweets.head()

,text
0,Republicans and Democrats have both created ou...
1,I was thrilled to be back in the Great city of...
2,RT @CBS_Herridge: READ: Letter to surveillance...
3,The Unsolicited Mail In Ballot Scam is a major...
4,RT @MZHemingway: Very friendly telling of even...


In [11]:
na_count = tweets.isnull().sum()
print("Number of NA rows:", na_count)

Number of NA rows: text    0
dtype: int64


In [12]:
print(tweets.count())

text    56571
dtype: int64


In [13]:
trump.text = trump.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)
trump.text = trump.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
trump.text = trump.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
trump = trump.loc[(trump.isRetweet == "f") & (trump.text != ""), :]
timestamps = trump.date.to_list()
tweets = trump.text.to_list()

In [14]:
tweets[0]

'republicans and democrats have both created our economic problems'

In [15]:
trump_tweets = pd.DataFrame()

In [16]:
trump_tweets['text'] = tweets

In [17]:
trump_tweets.head()

,text
0,republicans and democrats have both created ou...
1,i was thrilled to be back in the great city of...
2,the unsolicited mail in ballot scam is a major...
3,getting a little exercise this morning
4,thank you elise


In [18]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhava\AppData\Roaming\nltk_data...


True

In [19]:
filtered_text = []
lemmatizer = WordNetLemmatizer()

for w in tweets:
      filtered_text.append(lemmatizer.lemmatize(w))
print(filtered_text[:10])

['republicans and democrats have both created our economic problems', 'i was thrilled to be back in the great city of charlotte north carolina with thousands of hardworking american patriots who love our country cherish our values respect our laws and always put america first thank you for a wonderful evening kag', 'the unsolicited mail in ballot scam is a major threat to our democracy amp the democrats know it almost all recent elections using this system even though much smaller amp with far fewer ballots to count have ended up being a disaster large numbers of missing ballots amp fraud', 'getting a little exercise this morning', 'thank you elise', 'as per your request joe', 'the threshold identification of ballots is turning out to be even bigger than originally anticipated a very large number of ballots are impacted stay tuned', 'i m running as a proud democrat for the senate sleepy joe biden today it s only going to get worse it is not sustainable for our county china will own us'

In [21]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [22]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

In [23]:
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [24]:
vectorizer_model = CountVectorizer(stop_words="english")

In [25]:
ctfidf_model = ClassTfidfTransformer()

In [26]:
from bertopic.representation import MaximalMarginalRelevance
representation_model = MaximalMarginalRelevance(diversity=0.5)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [27]:
topic_model = BERTopic(
                embedding_model=embedding_model,    # Step 1 - Extract embeddings
                  umap_model=umap_model,              # Step 2 - Reduce dimensionality
                  hdbscan_model=hdbscan_model,        # Step 3 - Cluster reduced embeddings
                  vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
                  ctfidf_model=ctfidf_model,          # Step 5 - Extract topic words
                  representation_model=representation_model,
                  nr_topics=10                        # Step 6 - Diversify topic words
)

In [28]:
topics, probs = topic_model.fit_transform(filtered_text)

In [30]:
topics

[-1,
 -1,
 0,
 -1,
 1,
 -1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 -1,
 0,
 0,
 -1,
 0,
 -1,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 0,
 0,
 0,
 0,
 0,
 -1,
 -1,
 0,
 0,
 0,
 -1,
 0,
 0,
 -1,
 -1,
 0,
 0,
 -1,
 -1,
 0,
 0,
 0,
 -1,
 0,
 -1,
 0,
 -1,
 0,
 -1,
 1,
 -1,
 0,
 0,
 -1,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 2,
 -1,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 -1,
 -1,
 0,
 0,
 0,
 -1,
 1,
 -1,
 -1,
 0,
 -1,
 -1,
 0,
 -1,
 0,
 0,
 1,
 4,
 -1,
 -1,
 0,
 0,
 0,
 0,
 -1,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 2,
 0,
 0,
 0,
 -1,
 0,
 -1,
 -1,
 0,
 0,
 0,
 0,
 0,
 -1,
 -1,
 0,
 0,
 -1,
 -1,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 -1,
 -1,
 -1,
 0,
 -1,
 0,
 0,
 -1,
 1,
 0,
 0,
 0,
 -1,
 0,
 0,
 -1,
 0,
 1,
 0,
 0,
 -1,
 0,
 0,
 -1,
 -1,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 1,
 0,
 -1,
 -1,
 1,
 -1,
 -1,
 0,
 0,
 0,
 -1,
 0,
 -1,
 -1,
 0,
 -1,
 -1,
 2,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 0,
 -1,
 0,
 0,
 0,
 0,
 -1,
 0

In [31]:
topic_model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'amp | thank | china | vote | make', 21517],
                                   [1, 'thanks | happy | birthday | christmas | merry', 2875],
                                   [2, 'apprentice | celebapprentice | celebrity | miss | melania', 2500],
                                   [3, 'entrepreneurs | think | success | focus | passion', 1252],
                                   [4, 'amp | order | big | thank | law', 590],
                                   [5, 'wind | warming | turbines | windmills | birds', 391],
                                   [6, 'hunt | swamp | draining | continues | schneiderman', 73],
                                   [7, 'screen | apple | larger | mobile | samsung', 54],
                                   [8, 'pinehurst | golf | greens | critical | looked', 17]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([21517,  2875,  2500,  1252,   590,   391,    73,    54,    17],
                                       dtype=int64),
                         'sizemode': 'area',
                         'sizeref': 13.448125,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([-16.893408, -17.665543, -17.027573, -15.147218, -16.239996, -15.094373,
                          -16.819105, -16.544884, -17.27698 ], dtype=float32),
              'xaxis': 'x',
              'y': array([8.481073 , 4.016795 , 3.9122884, 8.421482 , 8.553059 , 7.182237 ,
                          6.2764106, 7.871214 , 3.3358324], dtype=float32),
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': -20.315373992919923,
                                'y': 6.33573746085167,
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': -16.57279541492462,
                                'xshift': 10,
                                'y': 9.836017417907716}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': -16.57279541492462,
                           'x1': -16.57279541492462,
                           'y0': 2.835457503795624,
                           'y1': 9.836017417907716},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': -20.315373992919923,
                           'x1': -12.83021683692932,
                           'y0': 6.33573746085167,
                           'y1': 6.33573746085167}],
               'sliders': [{'active': 0,
                            'pad': {'t': 50},
                            'steps': [{'args': [{'marker.color': [['red',
                                                                  '#B0BEC5',
                                                                  '#B0BEC5',
                                                                  '#B0BEC5',
                                                                  '#B0BEC5',
                                                                  '#B0BEC5',
                                                                  '#B0BE

In [32]:
topic_model.visualize_barchart()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.013656749575440998, 0.01448069628091833,
                    0.014676975160006038, 0.016886318911797822,
                    0.021324508681186665],
              'xaxis': 'x',
              'y': [make  , vote  , china  , thank  , amp  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0288634957483809, 0.04038531036098023, 0.09907068649174938,
                    0.11232075960755757, 0.3136301979973973],
              'xaxis': 'x2',
              'y': [merry  , christmas  , birthday  , happy  , thanks  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03450960044485094, 0.04355210009512805, 0.04379540791688855,
                    0.04417849719970491, 0.08200740415431565],
              'xaxis': 'x3',
              'y': [melania  , miss  , celebrity  , celebapprentice  , apprentice
                    ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.023968529750574924, 0.03322224900222096, 0.0398345851280326,
                    0.056793743770523326, 0.09632528017286539],
              'xaxis': 'x4',
              'y': [passion  , focus  , success  , think  , entrepreneurs  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.017920071387336207, 0.020525890432343163,
                    0.020785720311300243, 0.022822116127165063,
                    0.26577300946926935],
              'xaxis': 'x5',
              'y': [law  , thank  , big  , order  , amp  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.039184475626963554, 0.040400258429047806,
                    0.14131392997004436, 0.16935620714960475, 0.2046283627333394],
              'xaxis': 'x6',
              'y': [birds  , windmills  , turbines  , warming  , wind  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04261816156802776, 0.049154769368457134,
                    0.09365579166354761, 0.34854022438054844, 0.49718322416631977],
              'xaxis': 'x7',
              'y': [schneiderman  , continues  , draining  , swamp  , hunt  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.149834431716826, 0.15486983517124414, 0.1707693916966973,
                    0.27018360907079403, 0.28672091774025343],
              'xaxis': 'x8',
              'y': [samsung  , mobile  , larger  , apple  , screen  ],
              'yaxis': 'y8'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 1',
                                'x': 0.36250000000000004,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                       

In [33]:
topic_model.visualize_heatmap()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_amp_thank_china, 1_thanks_happy_birthday,
                    2_apprentice_celebapprentic..., 3_entrepreneurs_think_success,
                    4_amp_order_big, 5_wind_warming_turbines,
                    6_hunt_swamp_draining, 7_screen_apple_larger,
                    8_pinehurst_golf_greens],
              'xaxis': 'x',
              'y': [0_amp_thank_china, 1_thanks_happy_birthday,
                    2_apprentice_celebapprentic..., 3_entrepreneurs_think_success,
                    4_amp_order_big, 5_wind_warming_turbines,
                    6_hunt_swamp_draining, 7_screen_apple_larger,
                    8_pinehurst_golf_greens],
              'yaxis': 'y',
              'z': array([[1.        , 0.44142935, 0.66166781, 0.56391402, 0.50305082,
                           0.46801105, 0.43994705, 0.38835505, 0.21944903],
                          [0.44142935, 1.        , 0.51106748, 0.38643114, 0.39807048,
                           0.19238   , 0.19359957, 0.30024214, 0.22360607],
                          [0.66166781, 0.51106748, 1.        , 0.45903111, 0.46302634,
                           0.24696817, 0.26461245, 0.32381839, 0.33089494],
                          [0.56391402, 0.38643114, 0.45903111, 1.        , 0.39932261,
                           0.33953569, 0.2614399 , 0.33615598, 0.20214136],
                          [0.50305082, 0.39807048, 0.46302634, 0.39932261, 1.        ,
                           0.24462813, 0.14407613, 0.38005213, 0.15717779],
                          [0.46801105, 0.19238   , 0.24696817, 0.33953569, 0.24462813,
                           1.        , 0.31511294, 0.16612113, 0.27246283],
                          [0.43994705, 0.19359957, 0.26461245, 0.2614399 , 0.14407613,
                           0.31511294, 1.        , 0.13991309, 0.16890424],
                          [0.38835505, 0.30024214, 0.32381839, 0.33615598, 0.38005213,
                           0.16612113, 0.13991309, 1.        , 0.14108485],
                          [0.21944903, 0.22360607, 0.33089494, 0.20214136, 0.15717779,
                           0.27246283, 0.16890424, 0.14108485, 1.        ]])}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

In [34]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,16086,-1_amp_obama_big_like,"[amp, obama, big, like, make, fake, today, tha...",[thank you governor make america safe and grea...
1,0,21517,0_amp_thank_china_vote,"[amp, thank, china, vote, make, obama, news, l...","[make america great again, thank you trump, th..."
2,1,2875,1_thanks_happy_birthday_christmas,"[thanks, happy, birthday, christmas, merry, mo...","[thanks, thanks, thanks]"
3,2,2500,2_apprentice_celebapprentice_celebrity_miss,"[apprentice, celebapprentice, celebrity, miss,...",[the celebrity apprentice sunday night at pm o...
4,3,1252,3_entrepreneurs_think_success_focus,"[entrepreneurs, think, success, focus, passion...",[entrepreneurs don t sell yourself short don t...
5,4,590,4_amp_order_big_thank,"[amp, order, big, thank, law, think, trump, ru...","[law amp order, my int with on obama being all..."
6,5,391,5_wind_warming_turbines_windmills,"[wind, warming, turbines, windmills, birds, fa...","[wind turbines are a disaster, for those that ..."
7,6,73,6_hunt_swamp_draining_continues,"[hunt, swamp, draining, continues, schneiderma...","[witch hunt, witch hunt, witch hunt]"
8,7,54,7_screen_apple_larger_mobile,"[screen, apple, larger, mobile, samsung, tmobi...",[i can t believe apple isn t moving faster to ...
9,8,17,8_pinehurst_golf_greens_critical,"[pinehurst, golf, greens, critical, looked, pl...",[matt you don t understand golf even though yo...


In [35]:
probs_df=pd.DataFrame(probs)
probs_df['main percentage'] = pd.DataFrame({'max': probs_df.max(axis=1)})
probs_text=probs_df[["main percentage"]]
text_topics_assinged=pd.concat([full_train,probs_text],axis=1)

In [36]:
text_topics_assinged.head()

,text,main percentage
0,Republicans and Democrats have both created ou...,0.0
1,I was thrilled to be back in the Great city of...,0.0
2,RT @CBS_Herridge: READ: Letter to surveillance...,1.0
3,The Unsolicited Mail In Ballot Scam is a major...,0.0
4,RT @MZHemingway: Very friendly telling of even...,1.0


In [37]:
topic_labels = topic_model.generate_topic_labels(topic_prefix=False,nr_words=5,
                                                 separator=",")

In [38]:
topic_model.set_topic_labels(topic_labels)

In [39]:
topic_list = topic_model.get_topic_info()
topic_list

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,16086,-1_amp_obama_big_like,"amp,obama,big,like,make","[amp, obama, big, like, make, fake, today, tha...",[thank you governor make america safe and grea...
1,0,21517,0_amp_thank_china_vote,"amp,thank,china,vote,make","[amp, thank, china, vote, make, obama, news, l...","[make america great again, thank you trump, th..."
2,1,2875,1_thanks_happy_birthday_christmas,"thanks,happy,birthday,christmas,merry","[thanks, happy, birthday, christmas, merry, mo...","[thanks, thanks, thanks]"
3,2,2500,2_apprentice_celebapprentice_celebrity_miss,"apprentice,celebapprentice,celebrity,miss,melania","[apprentice, celebapprentice, celebrity, miss,...",[the celebrity apprentice sunday night at pm o...
4,3,1252,3_entrepreneurs_think_success_focus,"entrepreneurs,think,success,focus,passion","[entrepreneurs, think, success, focus, passion...",[entrepreneurs don t sell yourself short don t...
5,4,590,4_amp_order_big_thank,"amp,order,big,thank,law","[amp, order, big, thank, law, think, trump, ru...","[law amp order, my int with on obama being all..."
6,5,391,5_wind_warming_turbines_windmills,"wind,warming,turbines,windmills,birds","[wind, warming, turbines, windmills, birds, fa...","[wind turbines are a disaster, for those that ..."
7,6,73,6_hunt_swamp_draining_continues,"hunt,swamp,draining,continues,schneiderman","[hunt, swamp, draining, continues, schneiderma...","[witch hunt, witch hunt, witch hunt]"
8,7,54,7_screen_apple_larger_mobile,"screen,apple,larger,mobile,samsung","[screen, apple, larger, mobile, samsung, tmobi...",[i can t believe apple isn t moving faster to ...
9,8,17,8_pinehurst_golf_greens_critical,"pinehurst,golf,greens,critical,looked","[pinehurst, golf, greens, critical, looked, pl...",[matt you don t understand golf even though yo...


In [41]:
topic_list.to_excel('topic_list.xlsx')

In [ ]:
import pickle

In [ ]:
with open("Bertopic_model.pkl", "wb") as f:
    pickle.dump(topic_model, f)

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [42]:
tweets = pd.DataFrame({"Tweets": filtered_text,
                          "ID": range(len(filtered_text)),
                          "Topic": topics})
tweets_per_topic = tweets.groupby(['Topic'], as_index=False).agg({'Tweets': ' '.join})
cleaned_topics = topic_model._preprocess_text(tweets_per_topic.Tweets.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_topics]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)]
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words,
                                 texts=tokens,
                                 corpus=corpus,
                                 dictionary=dictionary,
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

In [43]:
print(coherence)

0.5807259485336854


In [44]:
def get_topics_with_coherence():
  topic_list.to_excel('topic_list.xlsx'),coherence

In [45]:
topic_model.save("topicmodel.pkl",serialization="pickle")

2024-03-06 00:13:44,545 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
